This project is to find a Chinese restaurant in New York.

In [1]:
import requests 
import pandas as pd
import numpy as np 
import random

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 


from IPython.display import Image 
from IPython.core.display import HTML 
    

from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium 

print('Folium installed')
print('Libraries imported.')



Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

   

In [26]:
CLIENT_ID = '' 
CLIENT_SECRET = ''
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [3]:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7149555 -74.0153365


In [15]:
search_query = 'Chinese'
radius = 500
print(search_query + ' .... OK!')

Chinese .... OK!


In [16]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=DETZL1BXN53YS30ZRFEPNCIJVKI3HV4C2JPMEGNRZOKL2AO5&client_secret=AO0GW1METSYLHYK4SKOR2GPGTZUBY2TM3X2DUE1VWG2PRTB5&ll=40.7149555,-74.0153365&v=20180604&query=Chinese&radius=500&limit=30'

In [17]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cf465cc4434b92169286dd7'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/streetfood_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d1cb941735',
      'name': 'Food Truck',
      'pluralName': 'Food Trucks',
      'primary': True,
      'shortName': 'Food Truck'}],
    'hasPerk': False,
    'id': '51c87762498e29ad0293dab1',
    'location': {'cc': 'US',
     'city': 'New York',
     'country': 'United States',
     'distance': 173,
     'formattedAddress': ['New York, NY', 'United States'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.713732,
       'lng': -74.016616}],
     'lat': 40.713732,
     'lng': -74.016616,
     'state': 'NY'},
    'name': 'Chinese Mirch (WTC Truck)',
    'referralId': 'v-1559520716'}]}}

In [18]:
venues = results['response']['venues']

In [19]:
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.state,name,referralId
0,"[{'name': 'Food Truck', 'shortName': 'Food Tru...",False,51c87762498e29ad0293dab1,US,New York,United States,173,"[New York, NY, United States]","[{'lng': -74.016616, 'label': 'display', 'lat'...",40.713732,-74.016616,NY,Chinese Mirch (WTC Truck),v-1559520716


In [20]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,state,id
0,Chinese Mirch (WTC Truck),Food Truck,US,New York,United States,173,"[New York, NY, United States]","[{'lng': -74.016616, 'label': 'display', 'lat'...",40.713732,-74.016616,NY,51c87762498e29ad0293dab1


In [27]:
dataframe_filtered.name

venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

venue_id = '51c87762498e29ad0293dab1'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url


In [25]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')



This venue has not been rated yet.


The venue has not been rated but it is the only option I found in the area!